In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states
from syft.client.syncing import resolve

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft/notebooks


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)
node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=dcfca9fb67b841c6b8e2c7a2fa940c57 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=5f166829f0774b6f892271f45ab4ad57 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  6.06it/s]


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 16.25it/s]

Uploading: numpy-data


# Make Requests

In [8]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("Computing mean...")
    return data.mean()


compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/3ec2a0853de1459c9b170127d87da9c2.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=2 image_uid=953acd67710b4eeb8b448d490b924e6d in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_607: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_607
override True
Computing mean...
SyftInfo: Landing the ephmeral node...


```python
Pointer
```
2.0

In [9]:
client_low_ds.code.request_code_execution(compute_mean)

syft.service.request.request.Request

## Sync to high side

In [10]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

SyftInfo: Node Landed!


In [11]:
low_state

syft.service.sync.sync_state.SyncState

In [12]:
diff_state = compare_states(low_state, high_state)

In [13]:
diff_state

syft.service.sync.diff_state.NodeDiff

In [14]:
resolved_state_low, resolved_state_high = resolve(diff_state, decision="low")

LOW SIDE BATCH STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #2a29e1b7573f46f9bfb6d5c89db879ad:
request_time: 2024-03-02 01:24:18
updated_at: None
status: RequestStatus.PENDING
changes: [
‎ ‎ ‎ ‎ syft.service.request.request.UserCodeStatusChange
]
requesting_user_verify_key: 9a90aa6be7a40651bf1bd57ffa360e4060a3022ab461cc66677454bcd2ca3029

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #63cbb57ff8114ef7840e9fe9e46e2c0f:
    service_func_name: compute_mean
    input_owners: [
    ‎ ‎ ‎ ‎ test_l
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]
    worker_pool_name: default-pool



HIGH SIDE BATCH STATE:



Decision: Syncing 2 objects from low side


LOW SIDE BATCH STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #63cbb57ff8114ef7840e9fe9e46e2c0f:
service_fun

In [15]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

Resolved state low side
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)

Resolved state high side
ResolvedSyncState(
  create_objs=[syft.service.request.request.Request, syft.service.code.user_code.UserCode, {NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [16]:
print(diff_state.diffs[-1].low_state)

NEW

class UserCode:
‎ ‎ ‎ ‎ service_func_name = compute_mean
‎ ‎ ‎ ‎ input_owners = [
‎ ‎ ‎ ‎ test_l
]
‎ ‎ ‎ ‎ code_status = [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]
‎ ‎ ‎ ‎ worker_pool_name = default-pool


In [17]:
client_low.apply_state(resolved_state_low)

SyftSuccess: Synced 0 items

In [18]:
resolved_state_high.create_objs[1]

```python
class UserCode
    id: UID = 63cbb57ff8114ef7840e9fe9e46e2c0f
    service_func_name: str = compute_mean
    shareholders: list = ['test_l']
    status: list = ['Node: test_l, Status: pending']
    
    code:

@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("Computing mean...")
    return data.mean()

```

In [19]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 3 items

# Run code high and sync back result

In [20]:
data_high = client_high.datasets[0].assets[0]

In [21]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = 719f7cc72bb1449f8d4bc2ed86bba423
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [22]:
# wait for the result
job_high.wait().get()

override True


02/03/24 01:24:26 FUNCTION LOG (719f7cc72bb1449f8d4bc2ed86bba423): Computing mean...


17.0

In [23]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [24]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)
assert isinstance(accept_res, SyftError)

In [25]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

Approving request for domain test_h
returning existing job
setting permission
None
None
Approving request for domain test_h
ADDING PERMISSION [READ: 1d861a07a1824fd389195f01aff596ef as 9a90aa6be7a40651bf1bd57ffa360e4060a3022ab461cc66677454bcd2ca3029] 1d861a07a1824fd389195f01aff596ef
New result 17.0
Job(719f7cc72bb1449f8d4bc2ed86bba423) Setting new result 1d861a07a1824fd389195f01aff596ef -> 1d861a07a1824fd389195f01aff596ef


SyftSuccess: Request submitted for updating result.

In [26]:
request.code.output_history[0]

```python
class ExecutionOutput:
  id: str = b0a0ccb770094f11b0501dd7f3203935
  created_at: str = 2024-03-02 01:24:05
  user_code_id: str = 63cbb57ff8114ef7840e9fe9e46e2c0f
  job_id: str = 719f7cc72bb1449f8d4bc2ed86bba423
  output_ids: str = [<LineageID: 1d861a07a1824fd389195f01aff596ef - 1534090569562199442>]

```

In [27]:
# accept_res = request.accept_by_depositing_result(job_info.result.get())

# assert isinstance(accept_res, SyftSuccess)
# accept_res

In [28]:
# Need to refresh Job because we overwrite result
job_high = client_high.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

## Sync back to low

In [29]:
client_high.jobs[0]

```python
class Job:
    id: UID = 719f7cc72bb1449f8d4bc2ed86bba423
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Computing mean...
JOB COMPLETED
    
```

In [30]:
low_state = client_low.sync.get_state()
high_state = client_high.sync.get_state()

high_state

syft.service.sync.sync_state.SyncState

In [31]:
diff_state_2 = compare_states(low_state, high_state)

diff_state_2

syft.service.sync.diff_state.NodeDiff

In [32]:
# syft absolute
from syft import ActionObject

In [33]:
resolved_state_low, resolved_state_high = resolve(
    diff_state_2, decision="high", share_private_objects=True
)

LOW SIDE BATCH STATE:

―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #2a29e1b7573f46f9bfb6d5c89db879ad:
node_uid: 8a1c04544655402190588aec30079bc3
changes: [
‎ ‎ ‎ ‎ syft.service.request.request.UserCodeStatusChange
]
history: [
]

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #63cbb57ff8114ef7840e9fe9e46e2c0f:
    service_func_name: compute_mean
    input_owners: [
    ‎ ‎ ‎ ‎ test_l
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]



HIGH SIDE BATCH STATE:

―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #2a29e1b7573f46f9bfb6d5c89db879ad:
node_uid: 8212e6797fde4c3fba4fc53ab555a886
changes: [
‎ ‎ ‎ ‎ syft.service.request.request.UserCodeStatusChange
‎ ‎ ‎ ‎ syft.service.request.request.ActionStoreChange
]
history: [
‎ ‎ ‎ ‎ syft.service.request.request.ChangeStatus
‎ ‎ 

In [34]:
print(resolved_state_low)
print()
print(resolved_state_high)

ResolvedSyncState(
  create_objs=[syft.service.output.output_service.ExecutionOutput, Pointer:
17.0, syft.service.job.job_stash.Job, syft.service.log.log.SyftLog],
  update_objs=[syft.service.request.request.Request, {NodeIdentity <name=test_h, id=8212e679, 🔑=d311a667>: (<UserCodeStatus.APPROVED: 'approved'>, '')}],
  delete_objs=[]
  new_permissions=[[READ: 1d861a07a1824fd389195f01aff596ef as 9a90aa6be7a40651bf1bd57ffa360e4060a3022ab461cc66677454bcd2ca3029], [READ: 719f7cc72bb1449f8d4bc2ed86bba423 as 9a90aa6be7a40651bf1bd57ffa360e4060a3022ab461cc66677454bcd2ca3029], [READ: 892d149a52c747f3b5e7e66c6a27964f as 9a90aa6be7a40651bf1bd57ffa360e4060a3022ab461cc66677454bcd2ca3029]]
)

ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [35]:
client_low.apply_state(resolved_state_low)

SyftSuccess: Synced 6 items

In [36]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 0 items

In [37]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

# Run code low

## Run

In [38]:
client_low_ds._fetch_api(client_low.credentials)

In [39]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low.get()

override False
output_history [syft.service.output.output_service.ExecutionOutput] 7973fdc25a2d48c1a3cbe2fea6c7baa8
output_history [syft.service.output.output_service.ExecutionOutput] 7973fdc25a2d48c1a3cbe2fea6c7baa8


17.0

In [40]:
res_low.get()

17.0

In [41]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id == job_high.result.id.id == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [42]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low.get()

override False
output_history [syft.service.output.output_service.ExecutionOutput] 7973fdc25a2d48c1a3cbe2fea6c7baa8
output_history [syft.service.output.output_service.ExecutionOutput] 7973fdc25a2d48c1a3cbe2fea6c7baa8


17.0

In [43]:
res_low.get()

17.0

In [44]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id == job_high.result.id.id == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [45]:
private_high.mean()

17.0

In [46]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
job_low

SyftWarning: There are existing jobs for this user code, returning the latest one

```python
class Job:
    id: UID = 719f7cc72bb1449f8d4bc2ed86bba423
    status: completed
    has_parent: False
    result: 17.0
    logs:

0 Log 892d149a52c747f3b5e7e66c6a27964f not available
JOB COMPLETED
    
```

In [47]:
job_low.wait().get()

17.0

In [48]:
job_low.logs()

Log 892d149a52c747f3b5e7e66c6a27964f not available


In [49]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)

## TODO

- Hard-deleting objects (in a clean way)
    - We *should* be able to delete without messing up a node state, because we make a single choice for a whole hierarchy 
    - What happens if we delete a queued/running job?
- Soft-delete/archive objects
    - More work, new flag on every object in every stash
- 